<a href="https://colab.research.google.com/github/manju838/Pytorch-Implementations/blob/main/pytorch_LeNet5_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch 
from torchvision import datasets, transforms 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import torch.nn as nn 
import torch.nn.functional as F 

In [3]:
mnist_train = datasets.MNIST(root = "",download = True, train = True,transform = transforms.ToTensor())
mnist_test = datasets.MNIST(root = "",download = True, train = False, transform = transforms.ToTensor())


Extracting MNIST/raw/train-images-idx3-ubyte.gz to MNIST/raw



Extracting MNIST/raw/train-labels-idx1-ubyte.gz to MNIST/raw



Extracting MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST/raw



Extracting MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST/raw
Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:479: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [4]:
mnist_trainloader = torch.utils.data.DataLoader(mnist_train, batch_size = 10, shuffle = True)
mnist_testloader = torch.utils.data.DataLoader(mnist_test, batch_size = 10, shuffle = True)

In [24]:
class LeNet(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(in_channels = 1, out_channels = 6, kernel_size = 5, stride=1)
    self.conv2 = nn.Conv2d(in_channels = 6, out_channels = 16, kernel_size = 5, stride=1)
    self.conv3 = nn.Conv2d(in_channels = 16, out_channels = 120, kernel_size = 5, stride=1)
    self.pool1 = nn.AvgPool2d(kernel_size = 2, stride=2)
    self.pool2 = nn.AvgPool2d(kernel_size = 2, stride=2)
    self.fc1 = nn.Linear(120,84)
    self.fc2 = nn.Linear(84,10)
  
  def forward(self, x):
    x = torch.tanh(self.pool1(torch.tanh(self.conv1(x))))
    x = self.conv3(torch.tanh(self.pool2(torch.tanh(self.conv2(x)))))
    x = torch.tanh(self.fc1(x.view(-1,120)))
    x = nn.Softmax(self.fc2(x))
    return(x)


In [25]:
x = torch.randn(64, 1, 32, 32)
model = LeNet()
model(x)

Softmax(
  dim=tensor([[ 1.1183e-01,  1.1243e-02,  2.9660e-02, -8.5716e-02,  1.2787e-02,
            4.1595e-02,  6.6436e-02, -8.0315e-02,  1.1355e-02, -3.5073e-02],
          [ 1.1652e-01,  1.4541e-02,  3.5791e-02, -7.9753e-02,  2.3147e-02,
            4.6768e-02,  7.6869e-02, -8.6405e-02,  9.0649e-03, -2.5391e-02],
          [ 1.2423e-01,  1.9364e-02,  2.8922e-02, -1.1488e-01,  5.5121e-03,
            2.6705e-02,  8.5089e-02, -1.0894e-01,  1.9203e-02, -2.1891e-02],
          [ 1.1429e-01,  1.3154e-02,  3.9950e-02, -1.0993e-01,  1.4614e-02,
            4.4306e-02,  7.0286e-02, -7.5060e-02, -1.3161e-03, -5.6706e-02],
          [ 1.3796e-01,  1.3896e-02,  4.4189e-02, -1.0562e-01,  1.0592e-02,
            4.6191e-02,  8.9830e-02, -1.0176e-01,  1.4904e-02, -1.7575e-02],
          [ 1.2296e-01,  2.7855e-02,  3.1735e-02, -9.7863e-02,  9.1696e-03,
            4.5337e-02,  7.2635e-02, -8.7407e-02,  8.8215e-03, -3.2071e-02],
          [ 1.3622e-01,  4.2037e-03,  2.5261e-02, -1.0248e-01, -9.550

In [17]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [18]:
model = LeNet().to(device=device)

In [19]:
model

LeNet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(16, 120, kernel_size=(5, 5), stride=(1, 1))
  (pool1): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (pool2): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (fc1): Linear(in_features=120, out_features=84, bias=True)
  (fc2): Linear(in_features=84, out_features=10, bias=True)
)

In [20]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

In [21]:
for epoch in range(10):
    for img,lbl in mnist_trainloader:
        img = img.to(device=device)
        lbl = lbl.to(device=device)
        y_pred = model(img)
        loss = criterion(y_pred, lbl)
        loss.backward() #Calculates the gradient
        optimizer.step() #Updates weights
        optimizer.zero_grad() #To prevet gradient accumulation
    
    if(epoch % 2 ==0): 
        model.eval()
        correct = 0
        total = 0
        for img,lbl in mnist_trainloader:
            img = img.to(device=device)
            lbl = lbl.to(device=device)
            y_pred = model(img)
            preds = torch.argmax(y_pred, 1)
            correct += (preds == lbl).sum()
            total += len(lbl)
        print(epoch, correct/total)


RuntimeError: ignored